In [1]:
from sklearn.datasets import make_blobs
import pandas as pdm
import numpy as np
import math
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yaml
import pandas as pd
import ast
from IPython.display import display

In [2]:
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute

In [3]:
# MEMO: top100 games
with open("C:\\Users\weejw\Documents\steamRecommendation\leanring\id_list.txt","r") as file:
    id_list = file.readlines()

app_ids = ast.literal_eval(id_list[0])

# date get from database

In [4]:
with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)
try:
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])

    #sql = "SELECT * FROM game_info"
    # MEMO: TOP 100 games
    game_info_sql = f"SELECT * FROM game_info where steam_appid in {str(tuple(app_ids))}"
    review_sql = f"SELECT * FROM review where steam_appid in {str(tuple(app_ids))}"
    
    game_info_rows = db_function.execute_select(game_info_sql)
    review_rows = db_function.execute_select(review_sql)
    
except:
    raise
    


In [5]:
len(game_info_rows), len(review_rows)

(95, 5649)

# game info 정제

In [6]:
game_info_cols = ['id', 'steam_appid', 'name', 'required_age', 'is_free', 'detailed_description', 'about_the_game',
       'short_description', 'developers', 'publishers', 'initial_price', 'final_price', 'discount_percent',
       'platforms', 'release_date', 'supported_languages', 'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id']

In [7]:
_game_info = pd.DataFrame(game_info_rows, columns=game_info_cols)

In [8]:
need_cols=['steam_appid','name','recommendations', 'categories_id', 'genres_id', 'mac', 'linux','windows', 'initial_price', 'final_price', 'discount_percent', 'supported_languages']

In [9]:
_game_info.columns

Index(['id', 'steam_appid', 'name', 'required_age', 'is_free',
       'detailed_description', 'about_the_game', 'short_description',
       'developers', 'publishers', 'initial_price', 'final_price',
       'discount_percent', 'platforms', 'release_date', 'supported_languages',
       'categories', 'genres', 'recommendations', 'categories_id',
       'genres_id'],
      dtype='object')

In [10]:
game_info = _game_info.fillna(0)

In [11]:
game_info['windows']=game_info['platforms'].apply(lambda x:1 if 'windows' in x else 0)
game_info['linux']=game_info['platforms'].apply(lambda x:1 if 'linux' in x else 0)
game_info['mac']=game_info['platforms'].apply(lambda x:1 if 'mac' in x else 0)

In [12]:
game_info['initial_price'] = game_info['initial_price'].apply(lambda x:int(x*0.01))
game_info['final_price'] = game_info['final_price'].apply(lambda x:int(x*0.01))
game_info['recommendations'] = game_info['recommendations'].apply(lambda x:int(x))

In [13]:
game_info = game_info[need_cols]

# review 정제

In [14]:
review_cols = ['id', 'steam_appid', 'steamid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review',
                             'last_played', 'votes_up', 'votes_funny', 'received_for_free', 'written_during_early_access',
                             'steam_purchase', 'num_reviews', 'playtime_forever', 'review_type', 'review']

In [15]:
_reviews = pd.DataFrame(review_rows, columns=review_cols)

In [16]:
_reviews

,id,steam_appid,steamid,num_games_owned,playtime_last_two_weeks,playtime_at_review,last_played,votes_up,votes_funny,received_for_free,written_during_early_access,steam_purchase,num_reviews,playtime_forever,review_type,review
0,16166,10,76561197967907928,125,0,6731,2018-11-25 20:56:35,2,0,0,0,1,27,8034,positive,말이 필요없음
1,16184,10,76561197973786146,86,0,10,2021-09-04 15:31:22,0,0,0,0,1,7,16,positive,This is where Valve has created the seed of ST...
2,16186,10,76561197974087280,40,0,20265,2014-09-08 23:51:44,0,0,0,0,1,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,16222,10,76561197985593224,94,0,9,2020-06-29 09:23:47,0,0,0,0,1,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,18713,10,76561197995131409,169,0,35,2020-02-14 20:56:10,0,0,0,0,1,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,16912,1468810,76561198031355345,34,1910,8610,2021-12-12 14:57:31,29,1,0,1,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,16914,1468810,76561198036523483,77,0,3510,2021-09-15 13:59:24,1,0,0,1,1,9,4466,positive,GOOD
5646,19498,1468810,76561198095212841,399,0,970,2021-01-31 04:26:58,6,4,0,1,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,19497,1468810,76561198203238402,183,0,29,2021-03-30 18:49:41,7,0,0,1,1,242,29,negative,add korean or english language so i can actua...


In [17]:
need_cols=['steam_appid', 'num_games_owned', 'playtime_last_two_weeks', 'playtime_at_review', 'votes_up', 'received_for_free', 'written_during_early_access', 'num_reviews', 
          'playtime_forever', 'review_type', 'review']

In [18]:
reviews = _reviews[need_cols]

In [19]:
reviews

,steam_appid,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


# Merga

In [20]:
all_data = pd.merge(game_info, reviews)

In [21]:
all_data

,steam_appid,name,recommendations,categories_id,genres_id,mac,linux,windows,initial_price,final_price,...,num_games_owned,playtime_last_two_weeks,playtime_at_review,votes_up,received_for_free,written_during_early_access,num_reviews,playtime_forever,review_type,review
0,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,125,0,6731,2,0,0,27,8034,positive,말이 필요없음
1,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,86,0,10,0,0,0,7,16,positive,This is where Valve has created the seed of ST...
2,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,40,0,20265,0,0,0,1,20265,positive,말이 필요 없는 한 시대를 풍미한 갓겜
3,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,94,0,9,0,0,0,84,9,positive,★★★★★ (5/5) - \n전설의 시작. \n후속작이 즐비함에도 세월을 잊고 아직...
4,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,...,169,0,35,0,0,0,41,66,negative,사람이 없습니다.\n윈도우 10에서 마우스 문제도 있는 듯.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,34,1910,8610,29,0,1,3,8834,positive,미완성이지만 한글패치를 만들어 https://cafe.naver.com/beemu ...
5645,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,77,0,3510,1,0,1,9,4466,positive,GOOD
5646,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,399,0,970,6,0,1,2,996,negative,刷新角色是 这游戏最好玩的部分。
5647,1468810,鬼谷八荒 Tale of Immortal,108235,"2,22,23,42","1,25,23,3,28,70",0,0,1,20500,20500,...,183,0,29,7,0,1,242,29,negative,add korean or english language so i can actua...


In [22]:
all_data.votes_up.max()

863

In [23]:
all_data.votes_up.mean()

7.303770578863515

In [24]:
all_data.columns

Index(['steam_appid', 'name', 'recommendations', 'categories_id', 'genres_id',
       'mac', 'linux', 'windows', 'initial_price', 'final_price',
       'discount_percent', 'supported_languages', 'num_games_owned',
       'playtime_last_two_weeks', 'playtime_at_review', 'votes_up',
       'received_for_free', 'written_during_early_access', 'num_reviews',
       'playtime_forever', 'review_type', 'review'],
      dtype='object')

# 1. 리뷰 수와 리뷰 찬반율과 추천 수 세가지를 이용해서

In [25]:
rate_table = pd.pivot_table(all_data[['name','review_type','review']], index='name', columns=['review_type'], aggfunc = ['count'])

In [26]:
rate_table = rate_table.droplevel(level=[0,1], axis=1)

In [27]:
rate_table.fillna(0, inplace=True)

In [28]:
rate_table['positive_rate'] = rate_table['positive'] / (rate_table['positive']+rate_table['negative'])

In [29]:
rate_table = rate_table.sort_values(by='positive_rate')

In [30]:
tmp_table = game_info[['name','recommendations']].set_index('name')

In [31]:
rate_table['recommendations'] = tmp_table['recommendations']

In [32]:
rate_table.sort_values(by=['positive_rate', 'recommendations'], ascending=False)[:10]

review_type,negative,positive,positive_rate,recommendations
name,,,,
Dirty Bomb®,0.0,13.0,1.000000,1348
Clicker Heroes,0.0,4.0,1.000000,804
Apex 레전드™,0.0,1.0,1.000000,694
Black Squad,0.0,1.0,1.000000,0
Euro Truck Simulator 2,2.0,75.0,0.974026,377751
Wallpaper Engine,2.0,37.0,0.948718,409858
Half-Life 2: Lost Coast,8.0,100.0,0.925926,7313
Fallout 4,2.0,23.0,0.920000,165669
Tomb Raider,1.0,11.0,0.916667,124604


# 2. votes_up 추가

In [33]:
new_review_data = all_data[['name','review_type','review','votes_up']]

In [34]:
new_review_data['new_review_count'] = 1 + (0.01 * new_review_data['votes_up'])

C:\Users\weejw\AppData\Local\Temp/ipykernel_17340/3710885618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_review_data['new_review_count'] = 1 + (0.01 * new_review_data['votes_up'])


In [35]:
new_rate_table = pd.pivot_table(new_review_data[['name','review_type','new_review_count']], index='name', columns=['review_type'], aggfunc = ['sum'])

In [36]:
new_rate_table = new_rate_table.droplevel(level=[0,1], axis=1)

In [37]:
new_rate_table['positive_rate'] = new_rate_table['positive'] / (new_rate_table['positive']+new_rate_table['negative'])

In [38]:
new_rate_table.fillna(0, inplace=True)

In [39]:
new_rate_table = new_rate_table.sort_values(by='positive_rate')

In [40]:
tmp_table = game_info[['name','recommendations']].set_index('name')

In [41]:
new_rate_table['recommendations'] = tmp_table['recommendations']

In [42]:
new_rate_table.sort_values(by=['positive_rate', 'recommendations'], ascending=False)[:10]

review_type,negative,positive,positive_rate,recommendations
name,,,,
Euro Truck Simulator 2,2.02,80.58,0.975545,377751
Wallpaper Engine,2.03,39.22,0.950788,409858
Half-Life 2: Lost Coast,8.10,100.73,0.925572,7313
Fallout 4,2.29,27.76,0.923794,165669
Tomb Raider,1.01,11.84,0.921401,124604
Valheim,7.42,82.75,0.917711,271164
Terraria,6.09,58.51,0.905728,692844
Phasmophobia,5.19,47.40,0.901312,299052
Monster Hunter: World,6.90,54.69,0.887969,208230


# 3.playtime

In [43]:
rate_table['steam_appid'] = game_info.set_index('name')['steam_appid']

In [44]:
rate_table

review_type,negative,positive,positive_rate,recommendations,steam_appid
name,,,,,
Street Warriors Online,2.0,0.0,0.000000,319,417910
SMITE®,2.0,0.0,0.000000,1325,386360
NBA 2K20,100.0,1.0,0.009901,45760,1089350
Rocket League®,98.0,1.0,0.010101,405107,252950
The Tiny Bang Story,18.0,1.0,0.052632,3192,96000
...,...,...,...,...,...
Euro Truck Simulator 2,2.0,75.0,0.974026,377751,227300
Dirty Bomb®,0.0,13.0,1.000000,1348,333930
Apex 레전드™,0.0,1.0,1.000000,694,1172470


In [45]:
mean_time=_reviews[['steam_appid','playtime_last_two_weeks','playtime_forever']].groupby('steam_appid').agg('mean')

In [46]:
pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['positive_rate','playtime_last_two_weeks','playtime_forever'], ascending=False)[:10]

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
90,Apex 레전드™,0.0,1.0,1.000000,694,1172470,4268.000000,18939.000000
91,Clicker Heroes,0.0,4.0,1.000000,804,363970,0.000000,28825.000000
89,Dirty Bomb®,0.0,13.0,1.000000,1348,333930,0.000000,5172.615385
92,Black Squad,0.0,1.0,1.000000,0,550650,0.000000,2340.000000
88,Euro Truck Simulator 2,2.0,75.0,0.974026,377751,227300,213.480519,7061.350649
87,Wallpaper Engine,2.0,37.0,0.948718,409858,431960,92.230769,4058.846154
86,Half-Life 2: Lost Coast,8.0,100.0,0.925926,7313,340,0.000000,53.990741
85,Fallout 4,2.0,23.0,0.920000,165669,377160,709.680000,18209.440000
84,Tomb Raider,1.0,11.0,0.916667,124604,203160,245.250000,1355.416667
83,Valheim,7.0,64.0,0.901408,271164,892970,256.042254,6529.000000


In [47]:
pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['playtime_forever','playtime_last_two_weeks','positive_rate'], ascending=False)[:10]

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
30,데스티니 가디언즈,13.0,13.0,0.500000,107412,1085660,877.692308,80428.076923
50,ARK: Survival Evolved,17.0,39.0,0.696429,384279,346110,1355.750000,74138.857143
53,Dota 2,14.0,38.0,0.730769,14399,570,119.769231,64015.057692
48,War Thunder,14.0,32.0,0.695652,1656,236390,173.782609,50289.173913
58,World of Warships,3.0,9.0,0.750000,747,552990,570.333333,47544.000000
5,Team Fortress 2,41.0,3.0,0.068182,12786,440,190.272727,45646.931818
70,Warframe,18.0,86.0,0.826923,2910,230410,116.259615,45377.019231
76,Sid Meier's Civilization® V,3.0,20.0,0.869565,110507,8930,499.000000,42932.130435
32,PUBG: BATTLEGROUNDS,92.0,101.0,0.523316,1548549,578080,607.694301,42808.435233
78,The Elder Scrolls V: Skyrim,1.0,7.0,0.875000,177932,72850,10.875000,42310.375000


In [48]:
res = pd.merge(rate_table.reset_index(), mean_time, on='steam_appid').sort_values(by=['playtime_last_two_weeks','playtime_forever','playtime_last_two_weeks'], ascending=False)[:10]

In [49]:
res

,name,negative,positive,positive_rate,recommendations,steam_appid,playtime_last_two_weeks,playtime_forever
90,Apex 레전드™,0.0,1.0,1.000000,694,1172470,4268.000000,18939.000000
24,New World,61.0,49.0,0.445455,179356,1063730,1531.454545,16398.454545
50,ARK: Survival Evolved,17.0,39.0,0.696429,384279,346110,1355.750000,74138.857143
42,Dead by Daylight,56.0,100.0,0.641026,402889,381210,1137.256410,39863.070513
40,Counter-Strike: Global Offensive,13.0,20.0,0.606061,3237267,730,1012.242424,32772.242424
49,Halo Infinite,7.0,16.0,0.695652,10596,1240440,892.739130,1456.956522
30,데스티니 가디언즈,13.0,13.0,0.500000,107412,1085660,877.692308,80428.076923
60,Rust,5.0,17.0,0.772727,572015,252490,875.000000,29759.272727
62,The Witcher® 3: Wild Hunt,8.0,29.0,0.783784,533212,292030,740.378378,12562.621622
85,Fallout 4,2.0,23.0,0.920000,165669,377160,709.680000,18209.440000


# fast text

In [50]:
text_data = _game_info[['name','categories', 'genres']]

In [51]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\weejw\PycharmProjects\steam_recommend\venv\Scripts\python.exe -m pip install --upgrade pip' command.


https://fasttext.cc/docs/en/crawl-vectors.html

In [52]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('get_data/cc.ko.300.bin.gz')

In [53]:
for w, sim in ko_model.wv.most_similar('코딩'):
    print(f'{w}: {sim}')

코딩과: 0.6641936898231506
코딩교육: 0.5640735030174255
코딩을: 0.554506242275238
코딩에: 0.5393441915512085
코딩의: 0.524505078792572
코딩은: 0.5197993516921997
코딩도: 0.5080924034118652
칸반: 0.5073744654655457
맵핑: 0.5020835995674133
프로그래밍: 0.48256731033325195


# 장르 이용하기

In [56]:
text_data

,name,categories,genres
0,Counter-Strike,"멀티플레이어,PvP,온라인 PvP,스크린 공유 및 분할 PvP,Valve Anti-...",액션
1,Day of Defeat,"멀티플레이어,Valve Anti-Cheat 사용",액션
2,Deathmatch Classic,"멀티플레이어,PvP,온라인 PvP,스크린 공유 및 분할 PvP,Valve Anti-...",액션
3,Half-Life: Opposing Force,"싱글 플레이어,멀티플레이어,Valve Anti-Cheat 사용,Remote Play...",액션
4,Ricochet,"멀티플레이어,PvP,온라인 PvP,Valve Anti-Cheat 사용",액션
...,...,...,...
90,Fall Guys: Ultimate Knockout,"멀티플레이어,MMO,PvP,온라인 PvP,협동,온라인 협동,Steam 도전 과제,컨...","액션,캐주얼,인디,대규모 멀티플레이어,스포츠"
91,Apex 레전드™,"멀티플레이어,PvP,온라인 PvP,협동,온라인 협동,Steam 도전 과제,컨트롤러 ...","액션,어드벤처,무료"
92,NARAKA: BLADEPOINT,"멀티플레이어,PvP,온라인 PvP,앱 내 구매,컨트롤러 일부 지원","액션,어드벤처,대규모 멀티플레이어"
93,Halo Infinite,"멀티플레이어,PvP,온라인 PvP,LAN PvP,플랫폼간 멀티플레이어,Steam 도...","액션,무료"


In [114]:
genres_type=set()
for genres in text_data['genres'].values:
    for genre in genres.split(","):
        genres_type.add(genre)
genres_type

{'RPG',
 '대규모 멀티플레이어',
 '디자인과 일러스트레이션',
 '레이싱',
 '무료',
 '사진 편집',
 '스포츠',
 '시뮬레이션',
 '앞서 해보기',
 '애니메이션과 모델링',
 '액션',
 '어드벤처',
 '유틸리티',
 '인디',
 '전략',
 '캐주얼'}

In [118]:
genre_table = text_data['name']

In [137]:
genre_matrix = []
for _, col in text_data[['name','genres']].iterrows():
    gmae_genre = col.genres.split(",")
    genre_one_hot = {'name':col['name']}
    
    for genre in genres_type:
        if genre in gmae_genre:
            genre_one_hot[genre] = 1
        else:
            genre_one_hot[genre] = 0
    
    genre_matrix.append(genre_one_hot)


In [146]:
one_hot_genre_matrix = pd.DataFrame(genre_matrix)

In [147]:
one_hot_genre_matrix

,name,어드벤처,대규모 멀티플레이어,디자인과 일러스트레이션,무료,시뮬레이션,사진 편집,액션,캐주얼,레이싱,유틸리티,앞서 해보기,전략,인디,스포츠,애니메이션과 모델링,RPG
0,Counter-Strike,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Day of Defeat,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Deathmatch Classic,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Half-Life: Opposing Force,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Ricochet,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Fall Guys: Ultimate Knockout,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0
91,Apex 레전드™,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
92,NARAKA: BLADEPOINT,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
93,Halo Infinite,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [202]:
target_matrix = np.array(one_hot_genre_matrix)[:,1:,]

In [203]:
target_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1]], dtype=object)

In [209]:
dist_res=dict()
standard_data =np.array(one_hot_genre_matrix)[-1][1:]
for idx, value in enumerate(target_matrix[:-1]):
    dist_res[idx] = np.linalg.norm(standard_data - value)

In [214]:
sort_res = sorted(dist_res.items(), key=lambda x:x[1])

In [216]:
sort_res[:5]

[(38, 1.0),
 (83, 1.0),
 (25, 1.4142135623730951),
 (30, 1.4142135623730951),
 (37, 1.4142135623730951)]

In [244]:
pd.DataFrame(text_data.iloc[-1][['name','genres']]).T

,name,genres
94,鬼谷八荒 Tale of Immortal,"액션,어드벤처,인디,RPG,시뮬레이션,앞서 해보기"


In [249]:
for info in sort_res[:5]:
    display(pd.DataFrame(text_data.iloc[info[0]][['name','genres']]).T)
    print(f"Distance:{info[1]}\n")

,name,genres
38,7 Days to Die,"액션,어드벤처,인디,RPG,시뮬레이션,전략,앞서 해보기"


Distance:1.0



,name,genres
83,Valheim,"액션,어드벤처,인디,RPG,앞서 해보기"


Distance:1.0



,name,genres
25,Terraria,"액션,어드벤처,인디,RPG"


Distance:1.4142135623730951



,name,genres
30,Grim Dawn,"액션,어드벤처,인디,RPG"


Distance:1.4142135623730951



,name,genres
37,The Forest,"액션,어드벤처,인디,시뮬레이션"


Distance:1.4142135623730951



# 카테고리 이용하기

In [250]:
text_data

,name,categories,genres
0,Counter-Strike,"멀티플레이어,PvP,온라인 PvP,스크린 공유 및 분할 PvP,Valve Anti-...",액션
1,Day of Defeat,"멀티플레이어,Valve Anti-Cheat 사용",액션
2,Deathmatch Classic,"멀티플레이어,PvP,온라인 PvP,스크린 공유 및 분할 PvP,Valve Anti-...",액션
3,Half-Life: Opposing Force,"싱글 플레이어,멀티플레이어,Valve Anti-Cheat 사용,Remote Play...",액션
4,Ricochet,"멀티플레이어,PvP,온라인 PvP,Valve Anti-Cheat 사용",액션
...,...,...,...
90,Fall Guys: Ultimate Knockout,"멀티플레이어,MMO,PvP,온라인 PvP,협동,온라인 협동,Steam 도전 과제,컨...","액션,캐주얼,인디,대규모 멀티플레이어,스포츠"
91,Apex 레전드™,"멀티플레이어,PvP,온라인 PvP,협동,온라인 협동,Steam 도전 과제,컨트롤러 ...","액션,어드벤처,무료"
92,NARAKA: BLADEPOINT,"멀티플레이어,PvP,온라인 PvP,앱 내 구매,컨트롤러 일부 지원","액션,어드벤처,대규모 멀티플레이어"
93,Halo Infinite,"멀티플레이어,PvP,온라인 PvP,LAN PvP,플랫폼간 멀티플레이어,Steam 도...","액션,무료"


In [252]:
category_type=set()
for categories in text_data['categories'].values:
    for category in categories.split(","):
        category_type.add(category)
category_type

{'LAN PvP',
 'LAN 협동',
 'MMO',
 'PvP',
 'Remote Play Together',
 'Steam 도전 과제',
 'Steam 순위표',
 'Steam 창작마당',
 'Steam 클라우드',
 'Steam 트레이딩 카드',
 'SteamVR 수집품',
 'TV용 Remote Play',
 'VR 지원',
 'Valve Anti-Cheat 사용',
 '공유/분할 화면',
 '레벨 에디터 포함',
 '멀티플레이어',
 '모바일용 Remote Play',
 '소스 SDK 포함',
 '스크린 공유 및 분할 PvP',
 '스크린 공유 및 분할 협동',
 '싱글 플레이어',
 '앱 내 구매',
 '온라인 PvP',
 '온라인 협동',
 '캡션 사용 가능',
 '컨트롤러 완벽 지원',
 '컨트롤러 일부 지원',
 '코멘터리 제공',
 '태블릿용 Remote Play',
 '통계',
 '플랫폼간 멀티플레이어',
 '협동'}

In [253]:
category_table = text_data['name']

In [259]:
category_matrix = []
for _, col in text_data[['name','categories']].iterrows():
    gmae_category = col.categories.split(",")
    category_one_hot = {'name':col['name']}
    
    for category in category_type:
        if category in gmae_category:
            category_one_hot[category] = 1
        else:
            category_one_hot[category] = 0
    
    category_matrix.append(category_one_hot)


In [260]:
one_hot_category_matrix = pd.DataFrame(category_matrix)

In [261]:
one_hot_category_matrix

,name,Valve Anti-Cheat 사용,Steam 순위표,레벨 에디터 포함,공유/분할 화면,소스 SDK 포함,앱 내 구매,모바일용 Remote Play,Steam 도전 과제,온라인 협동,...,싱글 플레이어,캡션 사용 가능,통계,온라인 PvP,컨트롤러 일부 지원,스크린 공유 및 분할 PvP,SteamVR 수집품,태블릿용 Remote Play,VR 지원,LAN PvP
0,Counter-Strike,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,Day of Defeat,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Deathmatch Classic,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,Half-Life: Opposing Force,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Ricochet,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Fall Guys: Ultimate Knockout,0,0,0,0,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,0
91,Apex 레전드™,0,0,0,0,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,0
92,NARAKA: BLADEPOINT,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
93,Halo Infinite,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [262]:
target_matrix = np.array(one_hot_category_matrix)[:,1:,]

In [263]:
target_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0]], dtype=object)

In [264]:
dist_res=dict()
standard_data =np.array(one_hot_category_matrix)[-1][1:]
for idx, value in enumerate(target_matrix[:-1]):
    dist_res[idx] = np.linalg.norm(standard_data - value)

In [265]:
sort_res = sorted(dist_res.items(), key=lambda x:x[1])

In [266]:
sort_res[:5]

[(20, 1.4142135623730951),
 (59, 1.4142135623730951),
 (44, 1.7320508075688772),
 (47, 1.7320508075688772),
 (50, 1.7320508075688772)]

In [270]:
pd.DataFrame(text_data.iloc[-1][['name','categories']]).T

,name,categories
94,鬼谷八荒 Tale of Immortal,"싱글 플레이어,Steam 도전 과제,Steam 클라우드,태블릿용 Remote Play"


In [272]:
for info in sort_res[:5]:
    display(pd.DataFrame(text_data.iloc[info[0]][['name','categories']]).T)
    print(f"Distance:{info[1]}\n")

,name,categories
20,Sid Meier's Civilization® V,"싱글 플레이어,멀티플레이어,Steam 도전 과제,Steam 트레이딩 카드,Steam..."


Distance:1.4142135623730951



,name,categories
59,Kathy Rain,"싱글 플레이어,Steam 도전 과제,Steam 트레이딩 카드,Steam 클라우드"


Distance:1.4142135623730951



,name,categories
44,Serena,싱글 플레이어


Distance:1.7320508075688772



,name,categories
47,A Story About My Uncle,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam 트레이딩 카드,S..."


Distance:1.7320508075688772



,name,categories
50,The Witcher® 3: Wild Hunt,"싱글 플레이어,Steam 도전 과제,컨트롤러 완벽 지원,Steam 트레이딩 카드,S..."


Distance:1.7320508075688772



In [274]:
game_info

,steam_appid,name,recommendations,categories_id,genres_id,mac,linux,windows,initial_price,final_price,discount_percent,supported_languages
0,10,Counter-Strike,116065,"1,49,36,37,8",1,1,1,1,10500,10500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 스페인어 스페인, 중국어 간체, 중국어 번..."
1,30,Day of Defeat,3055,"1,8",1,1,1,1,5500,5500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 스페인어 스페인"
2,40,Deathmatch Classic,1446,"1,49,36,37,8,44",1,1,1,1,5500,5500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 스페인어 스페인, 한국어, 러시아어, 중국..."
3,50,Half-Life: Opposing Force,10761,"2,1,8,44",1,1,1,1,5500,5500,0.0,"영어, 프랑스어, 독일어, 한국어"
4,60,Ricochet,2900,"1,49,36,8",1,1,1,1,5500,5500,0.0,"영어, 프랑스어, 독일어, 이탈리아어, 스페인어 스페인, 한국어, 러시아어, 중국..."
...,...,...,...,...,...,...,...,...,...,...,...,...
90,1097150,Fall Guys: Ultimate Knockout,361671,"1,20,49,36,9,38,22,28,35,41,42","1,4,23,29,18",0,0,1,20500,10250,50.0,"영어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 일본어, 한국어, 포르투..."
91,1172470,Apex 레전드™,694,"1,49,36,9,38,22,28,29,35","1,25,37",0,0,1,0,0,0.0,"영어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 일본어, 한국어, 폴란드..."
92,1203220,NARAKA: BLADEPOINT,68929,"1,49,36,35,18","1,25,29",0,0,1,22500,22500,0.0,"영어, 중국어 간체, 중국어 번체, 일본어, 한국어, 프랑스어, 독일어, 스페인어 ..."
93,1240440,Halo Infinite,10596,"1,49,36,47,27,22,28,35,23,43","1,37",0,0,1,0,0,0.0,"영어, 프랑스어, 이탈리아어, 독일어, 스페인어 스페인, 네덜란드어, 일본어, 한..."
